In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.metrics import CategoricalAccuracy
from keras.losses import MeanSquaredError
from keras.initializers import GlorotUniform

from importlib import reload

metrics = CategoricalAccuracy()

2023-06-07 23:22:55.362855: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 23:22:56.376340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-07 23:22:58.944290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-06-07 23:22:58.944337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: wentao-ubuntu
2023-06-07 23:22:58.944347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: wentao-ubuntu
2023-06-07 23:22:58.944480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version 

In [2]:
# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

In [3]:
batch_size = 1
hidden_units = 256
epoch = 1
function_num = 5
table_num = 6

dropout_probability = (1-1/2**function_num)**table_num

# The function can fit a model and return accuracy of test data
def f(x_train, y_train, x_test, y_test, model, batch_size, epoches, weight = None, progress = True, reset = True):
    metrics = CategoricalAccuracy()
    if(weight is not None):
        model.setWeight(weight)
    model.fit(x_train, y_train, 0.01, epoches = epoches, batch_size = batch_size, progress = progress)
    y_hat = model.predict(x_test)
    metrics.reset_state()
    metrics.update_state(y_test, y_hat)
    if(reset):
        model.reset()
    return metrics.result().numpy()

In [8]:
import NeuralNetwork as nn
# Not divide by rate
LSH1 = nn.MyModel()
LSH1.addLayer(nn.MyLayer(input_size, hidden_units, dropout_lsh = True, function_num=function_num, table_num=table_num))
LSH1.addLayer(nn.MyLayer(hidden_units, num_labels))
# LSH dropout
#print("Deep Learning with LSH Dropout")
#print("\nAccuracy: " + str(f(x_train, y_train, x_test, y_test, LSH1, batch_size, epoches = epoch, progress = True, reset=False)))
accuracy_LSH1 = [0 for i in range(10)]
for i in range(10):
    accuracy_LSH1[i] = f(x_train, y_train, x_test, y_test, LSH1, batch_size, epoches = epoch, progress = False, reset=False)
    LSH1.reset()

In [9]:
import NeuralNetwork1 as nn1
# Divide by rate
LSH2 = nn1.MyModel()
LSH2.addLayer(nn1.MyLayer(input_size, hidden_units, dropout_lsh = True, function_num=function_num, table_num=table_num))
LSH2.addLayer(nn1.MyLayer(hidden_units, num_labels))
# LSH dropout
#print("Deep Learning with LSH Dropout")
#print("\nAccuracy: " + str(f(x_train, y_train, x_test, y_test, LSH2, batch_size, epoches = epoch, progress = True, reset=False)))

accuracy_LSH2 = [0 for i in range(10)]
for i in range(10):
    accuracy_LSH2[i] = f(x_train, y_train, x_test, y_test, LSH2, batch_size, epoches = epoch, progress = False, reset=False)
    LSH2.reset()

/home/wentao/Documents/NeuralNetwork/NeuralNetwork1.py:147: RuntimeWarning: invalid value encountered in double_scalars
  e = np.sqrt(1-(norm/self.max_norm)**2)
/home/wentao/Documents/NeuralNetwork/NeuralNetwork1.py:77: RuntimeWarning: invalid value encountered in multiply
  dy = (dy*self.mask)/(self.rate)


In [21]:
np.mean(accuracy_LSH2)

0.84231246

In [19]:
del accuracy_LSH2[8]

In [15]:
np.mean(accuracy_LSH1)

0.84312993